In [3]:
# Required imports
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QFileDialog, QVBoxLayout, QGraphicsView, QGraphicsScene, QTextBrowser, QScrollArea, QLineEdit, QLabel, QHBoxLayout, QGridLayout, QCheckBox, QTextEdit, QMessageBox
from PyQt5.QtGui import QPen, QColor
from PyQt5.QtCore import Qt
import ezdxf
import os
import qdarktheme
import numpy as np
import decimal

# Parses input file to extract entities relative to gcode
def parse_dxf_file(file_path):
    parsed_data = []
    doc = ezdxf.readfile(file_path)
    for entity in doc.modelspace():
        if entity.dxftype() == 'LINE':
            start_point = entity.dxf.start
            end_point = entity.dxf.end
            parsed_data.append({
                'type': 'LINE',
                'start_point': (start_point.x, start_point.y),
                'end_point': (end_point.x, end_point.y)
            })
        elif entity.dxftype() == 'ARC':
            center = entity.dxf.center
            radius = entity.dxf.radius
            start_angle = entity.dxf.start_angle
            end_angle = entity.dxf.end_angle
            parsed_data.append({
                'type': 'ARC',
                'center_point': (center.x, center.y),
                'radius': radius,
                'start_angle': start_angle,
                'end_angle': end_angle
            })
        elif entity.dxftype() == 'LWPOLYLINE':
            polyline_data = []
            for vertex in entity.points():
                polyline_data.append((vertex.x, vertex.y))
            parsed_data.append({
                'type': 'POLYLINE',
                'vertices': polyline_data
            })

    return parsed_data

# block num space padding logic
def blockNumPad(blockNum, GcodeTrue):
    blockNumStr = ""
    if blockNum < 10:
        blockNumStr = "    0" + str(blockNum)
    elif blockNum < 100:
        blockNumStr = "    " + str(blockNum)
    else:
        blockNumStr = "   " + str(blockNum)
        
    if GcodeTrue:
        blockNumStr = blockNumStr + " "
    return blockNumStr

# Adjusts x coordinate based on stock radius, Lathe Z axis in implementation
def compX(xcoordinate, stockRadius):
    return xcoordinate - stockRadius

# returns a formated G01 command
def formatG00G01(xcord, ycord, gcodeType):
    # Set the precision (number of decimal places)
    decimal.getcontext().prec = 2  # Change this to your desired precision

    # Set the rounding mode (optional, default is ROUND_HALF_EVEN)
    decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    
    if gcodeType == "G00":
        # Both coordinates are negative
        if xcord < 0 and ycord < 0:
            return f'00 -{abs(int(xcord*100)):04} -{abs(int(ycord*100)):05}'
        # Only X neg
        elif xcord < 0 and ycord >= 0:
            return f'00 -{abs(int(xcord*100)):04}  {abs(int(ycord*100)):05}'
        # Only y neg    
        elif xcord >= 0 and ycord < 0:
            return f'00  {abs(int(xcord*100)):04} -{abs(int(ycord*100)):05}'
        # Both positive
        elif xcord >= 0 and ycord >=0:
            return f'00  {abs(int(xcord*100)):04}  {abs(int(ycord*100)):05}'
    if gcodeType == "G01":
        # Both coordinates are negative
        if xcord < 0 and ycord < 0:
            return f'01 -{abs(int(xcord*100)):04} -{abs(int(ycord*100)):05}'
        # Only X neg
        elif xcord < 0 and ycord >= 0:
            return f'01 -{abs(int(xcord*100)):04}  {abs(int(ycord*100)):05}'
        # Only y neg    
        elif xcord >= 0 and ycord < 0:
            return f'01  {abs(int(xcord*100)):04} -{abs(int(ycord*100)):05}'
        # Both positive
        elif xcord >= 0 and ycord >=0:
            return f'01  {abs(int(xcord*100)):04}  {abs(int(ycord*100)):05}'

        
# Formats feedrate for gcode output
def formatFeed(feedrate):
    return f' {feedrate:03}'
    
# Sorts data being parsed from DXF so lines are in consecutive order
# Lines are in order starting from Z0 which should be where the 
#   start of the cut occurs
def sortParsedData(parsed_data):
    # Create a dictionary to map each endpoint to its corresponding line
    endpoint_to_line = {(line['start_point'][0], line['start_point'][1]): line for line in parsed_data}

    # Find the starting point (0)
    first_line = None
    for line in parsed_data:
        if line['start_point'][0] == 0:
            first_line = line
            break

    # Initialize the sorted list with the first line
    sorted_lines = [first_line]
    endpoint = first_line['end_point']

    # Continue to find and add consecutive lines
    while True:
        next_line = endpoint_to_line.get(endpoint)
        if next_line is None:
            break
        sorted_lines.append(next_line)
        endpoint = next_line['end_point']
        
    return sorted_lines
        
# Parses DXF data into Emco supported GCode
def generate_gcode_from_dxf(parsed_data, isUseM3M5Checked, isStartRetractXChecked,isStartRetractZChecked, isStartRetractXZChecked, stockRadius, roughFeed):
    
    # starting blocks
    gcode = []
    blockNum = 0
    gcode.append(f'%\n')
    gcode.append(f'    N` G`   X `    Z `  F`  H\n')
    
    # Check if using m3/m5
    def usingM3M5(blockNum):
        if isUseM3M5Checked:
            gcode.append(f'{blockNumPad(blockNum, 0)}M03\n')
            blockNum +=1
        return blockNum
    
    # Check stating retracts
    if isStartRetractXChecked:
        # The "Retract before start in X" checkbox is checked
        gcode.append(f'{blockNumPad(blockNum, 1)}00    50\n')
        blockNum += 1
        blockNum = usingM3M5(blockNum)
        gcode.append(f'{blockNumPad(blockNum, 1)}00 -  50\n')
        blockNum += 1
    elif isStartRetractZChecked:
        # The "Retract bblockNumefore start in Z" checkbox is checked
        gcode.append(f'{blockNumPad(blockNum, 1)}00           50\n')
        blockNum += 1
        blockNum = usingM3M5(blockNum)
        gcode.append(f'{blockNumPad(blockNum, 1)}00       -   50\n')
        blockNum += 1
    elif isStartRetractXZChecked:
        # The "ReblockNumtract before start in X and Z" checkbox is checked
        gcode.append(f'{blockNumPad(blockNum, 1)}00    50     50\n')
        blockNum += 1
        blockNum = usingM3M5(blockNum)
        gcode.append(f'{blockNumPad(blockNum, 1)}00 -  50 -   50\n')
        blockNum += 1
    else:
        blockNum = usingM3M5(blockNum)

    # Move to first cut
    parsed_data = sortParsedData(parsed_data)
    firstEndPoint = (parsed_data[0]['start_point'][0], parsed_data[0]['start_point'][1])
    toAppend = (f'{blockNumPad(blockNum, 1)}{formatG00G01(compX(firstEndPoint[1], stockRadius), firstEndPoint[0], "G01")}{formatFeed(roughFeed)}\n')
    gcode.append(toAppend)
    blockNum += 1
    current_x = compX(firstEndPoint[1], stockRadius)
    current_y = firstEndPoint[0]
    
    # generate main gcode blocks
    for entity in parsed_data:
        toAppend = ""
        if entity['type'] == 'LINE':
            end_x, end_y = entity['end_point']
            gotoX = compX(end_y, stockRadius) - current_x
            current_x = compX(end_y, stockRadius)   
            gotoY = end_x - current_y
            current_y = end_x
            gcodeToAdd = (f'{formatG00G01(gotoX, gotoY, "G01")}{formatFeed(roughFeed)}')
        elif entity['type'] == 'ARC':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            start_angle = entity['start_angle']
            end_angle = entity['end_angle']
            if start_angle < end_angle:
                gcodeToAdd = (f'M02 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f} A{start_angle:.3f} B{end_angle:.3f}')
            else:
                gcodeToAdd = (f'M03 X{center_x:.3f} Y{center_y:.3f} R{radius:.3f} A{start_angle:.3f} B{end_angle:.3f}')
        elif entity['type'] == 'POLYLINE':
            vertices = entity['vertices']
            for vertex in vertices:
                x, y = vertex
                gcodeToAdd = (f'01 X{x:.3f} Y{y:.3f}')
            
        
        # append to output
        toAppend = blockNumPad(blockNum, 1) + gcodeToAdd + "\n"
        gcode.append(toAppend)
        blockNum += 1
           
    # final retract to beginning of cut
    toAppend = (f'{blockNumPad(blockNum, 1)}{formatG00G01(-current_x, 0, "G00")}\n')
    gcode.append(toAppend)
    blockNum += 1
    toAppend = (f'{blockNumPad(blockNum, 1)}{formatG00G01(0, -current_y, "G00")}\n')
    gcode.append(toAppend)
    blockNum += 1
            
    # ending blocks
    if isUseM3M5Checked:
        gcode.append(f'{blockNumPad(blockNum, 0)}M05\n')
        blockNum +=1
    
    # File end
    gcode.append(f'{blockNumPad(blockNum, 0)}M30\n')
    
    # MFI end input
    gcode.append(f'   M\n')
    return gcode

# Determins the max DXF size for scaling DXF preview 
def calculate_drawing_extents(entities):
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')

    for entity in entities:
        if entity['type'] == 'LINE':
            start_x, start_y = entity['start_point']
            end_x, end_y = entity['end_point']
            min_x = min(min_x, start_x, end_x)
            max_x = max(max_x, start_x, end_x)
            min_y = min(min_y, start_y, end_y)
            max_y = max(max_y, start_y, end_y)
        elif entity['type'] == 'CIRCLE':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            min_x = min(min_x, center_x - radius)
            max_x = max(max_x, center_x + radius)
            min_y = min(min_y, center_y - radius)
            max_y = max(max_y, center_y + radius)
        elif entity['type'] == 'ARC':
            center_x, center_y = entity['center_point']
            radius = entity['radius']
            start_angle = entity['start_angle']
            end_angle = entity['end_angle']
            min_x = min(min_x, center_x - radius)
            max_x = max(max_x, center_x + radius)
            min_y = min(min_y, center_y - radius)
            max_y = max(max_y, center_y + radius)
        elif entity['type'] == 'POLYLINE':
            for vertex in entity['vertices']:
                x, y = vertex
                min_x = min(min_x, x)
                max_x = max(max_x, x)
                min_y = min(min_y, y)
                max_y = max(max_y, y)

    return min_x, min_y, max_x, max_y

# Emco Processor GUI
class DXFParserGUI(QWidget):
    
    def __init__(self):
        super().__init__()
        self.initUI()
        self.file_path = ""
        self.output_code = ""

    def initUI(self):
        layout = QVBoxLayout()

        # Display the DXF drawing
        self.view = QGraphicsView(self)
        self.scene = QGraphicsScene()
        self.view.setScene(self.scene)
        layout.addWidget(self.view)

        # Open DXF file button
        self.btn_open = QPushButton("Open DXF File")
        self.btn_open.clicked.connect(self.openFile)
        layout.addWidget(self.btn_open)

        # Create a grid layout for labels and text boxes
        grid_layout = QGridLayout()

        # Stock Radius input
        self.stock_radius_label = QLabel("Stock Radius (mm):")
        self.stock_radius_input = QLineEdit()
        grid_layout.addWidget(self.stock_radius_label, 0, 0)
        grid_layout.addWidget(self.stock_radius_input, 0, 1)

        # Roughing Feedrate input
        self.roughing_feedrate_label = QLabel("Roughing Feedrate (mm/min):")
        self.roughing_feedrate_input = QLineEdit()
        grid_layout.addWidget(self.roughing_feedrate_label, 0, 3)
        grid_layout.addWidget(self.roughing_feedrate_input, 0, 4)

        # Roughing Stepdown input
        self.roughing_stepdown_label = QLabel("Roughing Stepdown (mm):")
        self.roughing_stepdown_input = QLineEdit()
        grid_layout.addWidget(self.roughing_stepdown_label, 0, 5)
        grid_layout.addWidget(self.roughing_stepdown_input, 0, 6)

        # Finishing Feedrate input
        self.finishing_feedrate_label = QLabel("Finishing Feedrate (mm/min):")
        self.finishing_feedrate_input = QLineEdit()
        grid_layout.addWidget(self.finishing_feedrate_label, 0, 7)
        grid_layout.addWidget(self.finishing_feedrate_input, 0, 8)

        # Finishing Stepdown input
        self.finishing_stepdown_label = QLabel("Finishing Stepdown (mm):")
        self.finishing_stepdown_input = QLineEdit()
        grid_layout.addWidget(self.finishing_stepdown_label, 0, 9)
        grid_layout.addWidget(self.finishing_stepdown_input, 0, 10)
        
        # Use M3/M5 toggle button
        self.use_m3_m5_checkbox = QCheckBox("Use M3/M5")
        grid_layout.addWidget(self.use_m3_m5_checkbox, 1, 0, 1, 1)
        
        # Retract in X, Z, or XZ toggle button 
        self.retract_start_x_checkbox = QCheckBox("Retract before start in X")
        grid_layout.addWidget(self.retract_start_x_checkbox, 1, 2, 1, 3)
        self.retract_start_z_checkbox = QCheckBox("Retract before start in Z")
        grid_layout.addWidget(self.retract_start_z_checkbox, 1, 4, 1, 5)
        self.retract_start_xz_checkbox = QCheckBox("Retract before start in X and Z")
        grid_layout.addWidget(self.retract_start_xz_checkbox, 1, 6, 1, 7)

        
        # Create a list of the checkboxes for easier management
        retract_checkboxes = [
            self.retract_start_x_checkbox,
            self.retract_start_z_checkbox,
            self.retract_start_xz_checkbox
        ]
        
        # Connect the checkboxes to the mutually exclusive function
        for checkbox in retract_checkboxes:
            checkbox.clicked.connect(lambda state, c=checkbox: self.exclusive_retract_behavior(c, retract_checkboxes))
    
        # Add the grid layout to the main layout
        layout.addLayout(grid_layout)

        # Scrollable G-code display that allows user to edit directly
        self.gcode_browser = QTextEdit(self)
        self.gcode_browser_cursor = self.gcode_browser.textCursor()
        self.scroll_area = QScrollArea(self)
        self.scroll_area.setWidget(self.gcode_browser)
        layout.addWidget(self.scroll_area)
        self.scroll_area.setWidgetResizable(True)
        
        # Set the font size for the QTextBrowser
        font = self.gcode_browser.currentFont()
        font.setPointSize(14)  # Adjust the font size (in points) as needed
        self.gcode_browser.setFont(font)
        
        # Create a grid layout for generate and save buttons
        grid_layout3 = QGridLayout()
        
        # Add M00 button
        self.btn_M00 = QPushButton("Add M00")
        self.btn_M00.clicked.connect(self.insertM00)
        grid_layout3.addWidget(self.btn_M00, 0, 0)
        
        # Add M00 button
        self.btn_G21 = QPushButton("Add G21")
        self.btn_G21.clicked.connect(self.insertG21)
        grid_layout3.addWidget(self.btn_G21, 0, 1)
        
        # Finishing Stepdown input
        self.addM00_label = QLabel("Add at block number:")
        self.addM00G21_input = QLineEdit()
        self.addM00G21_input.setMaximumWidth(200)
        grid_layout3.addWidget(self.addM00_label, 0, 2)
        grid_layout3.addWidget(self.addM00G21_input, 1, 2)

        # Generate G-code button
        self.btn_gen = QPushButton("Generate G-code")
        self.btn_gen.clicked.connect(self.generateGCode)
        grid_layout3.addWidget(self.btn_gen, 0, 4)
        
        # Save G-code button
        self.btn_save = QPushButton("Save G-code")
        self.btn_save.clicked.connect(self.saveGCode)
        grid_layout3.addWidget(self.btn_save, 0, 5)
        
        # Add the grid layout to the main layout
        layout.addLayout(grid_layout3)

        self.setLayout(layout)
        self.setGeometry(100, 100, 1600, 1600)
        self.setWindowTitle("DXF Parser")
        self.show()
        
##############################################################################################
#################################  CALLBACK FUNCTIONS  #######################################
##############################################################################################

    # Function to read stock radius input
    def getStockRadius(self):
        if self.stock_radius_input.text() != "":
            return int(self.stock_radius_input.text())
        else:
            return ""
        
    # Function to read roughing feedrate
    def getRoughFeed(self):
        if self.roughing_feedrate_input.text() != "":
            return int(self.roughing_feedrate_input.text())
        else:
            return ""    
    
    # Function to get the value of the Use M3/M5 checkbox
    def isUseM3M5Checked(self):
        return self.use_m3_m5_checkbox.isChecked()
    
    # Functions to get the value of the start retract boxes
    def isStartRetractXChecked(self):
        return self.retract_start_x_checkbox.isChecked()
    
    def isStartRetractZChecked(self):
        return self.retract_start_z_checkbox.isChecked()
    
    def isStartRetractXZChecked(self):
        return self.retract_start_xz_checkbox.isChecked()
    
    # Insert M00 at cursor position in gcode output
    def insertM00(self):
        self.insertBlock("M00")
        
    # Insert 21 at cursor position in gcode output
    def insertG21(self):
        self.insertBlock("G21")
    
    # Insert M00, G21 at cursor position in gcode output
    def insertBlock(self, M00G21):
        
        # Insert M00 or popup error if block insertion number has been added
        currentBlock = self.addM00G21_input.text()
        if self.gcode_browser.toPlainText() == "":
            self.errorMessage("Please generate GCode first")
        elif currentBlock != "":
            currentBlock = int(currentBlock)
            text_to_insert = f"    {currentBlock:02d}  {M00G21}"  # Format the text
        
            # Get the entire text from the QTextEdit
            full_text = self.gcode_browser.toPlainText()

            # Split the text into lines
            lines = full_text.split('\n')

            # Update the block numbers in the lines
            updated_lines = []
            block_number = 0  # The starting block number
            for line in lines:
                if line.strip().startswith("M") or line.strip().startswith("%") or line.strip().startswith("N") or line == "":  # Check for the start/end of the file
                    updated_lines.append(line)  # Keep the "M" line
                elif int(line[:6]) == currentBlock: 
                    updated_lines.append(text_to_insert)
                    block_number += 1
                    updated_line = f"    {block_number:02d}{line[6:]}"  # Update the block number
                    updated_lines.append(updated_line)
                    block_number += 1
                else:
                    updated_line = f"    {block_number:02d}{line[6:]}"  # Update the block number
                    updated_lines.append(updated_line)
                    block_number += 1

                # Join the updated lines into a single string
                updated_text = '\n'.join(updated_lines)

                # Set the updated text back into the QTextEdit
                self.gcode_browser.setPlainText(updated_text)
        else:
            self.errorMessage("You need to enter a block number for insertion position")

    # Mutually exclusive function
    def exclusive_retract_behavior(self, clicked_checkbox, checkboxes):
        for checkbox in checkboxes:
            if checkbox is not clicked_checkbox:
                checkbox.setChecked(False)
    
    # Opens DXF file from your computer
    def openFile(self):
        options = QFileDialog.Options()
        self.file_path, _ = QFileDialog.getOpenFileName(self, "Open DXF File", "", "DXF Files (*.dxf);;All Files (*)", options=options)
        if self.file_path:
            entities = parse_dxf_file(self.file_path)
            self.parseAndDisplayDXF(entities)
           
    # Starts gcode generating process
    def generateGCode(self):
        if self.file_path == "":
            self.errorMessage("You need to insert a DXF first")
        elif self.getStockRadius() == "":
            self.errorMessage("You need to enter a stock radius")
        elif self.getRoughFeed() == "":
            self.errorMessage("You need to enter a roughing feedrate")
        else:
            entities = parse_dxf_file(self.file_path)
            self.output_code = generate_gcode_from_dxf(entities, self.isUseM3M5Checked(), self.isStartRetractXChecked(), self.isStartRetractZChecked(), self.isStartRetractXZChecked(), self.getStockRadius(), self.getRoughFeed())
            self.gcode_browser.clear()
            self.gcode_browser.append(''.join(self.output_code))
            
    # Displays DXF and scales uniformly to fit screen
    def parseAndDisplayDXF(self, entities):
        self.scene.clear()
        # Get the drawing extents for scaling
        min_x, min_y, max_x, max_y = calculate_drawing_extents(entities)
        drawing_width = max_x - min_x
        drawing_height = max_y - min_y
        scale_factor = 750 / max(drawing_width, drawing_height)  # Adjust the scale as needed

        for entity in entities:
            if entity['type'] == 'LINE':
                start_x, start_y = entity['start_point']
                end_x, end_y = entity['end_point']
                line = self.scene.addLine(start_x * scale_factor, -start_y * scale_factor, end_x * scale_factor, -end_y * scale_factor)
                pen = QPen(QColor(0, 0, 255))
                line.setPen(pen)
            elif entity['type'] == 'CIRCLE':
                center_x, center_y = entity['center_point']
                radius = entity['radius']
                circle = self.scene.addEllipse((center_x - radius) * scale_factor, (-center_y - radius) * scale_factor, radius * 2 * scale_factor, radius * 2 * scale_factor)
                pen = QPen(QColor(0, 0, 255))
                circle.setPen(pen)
            elif entity['type'] == 'POLYLINE':
                vertices = entity['vertices']
                polyline = QGraphicsView()
                poly_scene = QGraphicsScene()
                polyline.setScene(poly_scene)
                pen = QPen(QColor(0, 0, 255))
                for i in range(len(vertices) - 1):
                    start_x, start_y = vertices[i]
                    end_x, end_y = vertices[i + 1]
                    line = poly_scene.addLine(start_x * scale_factor, -start_y * scale_factor, end_x * scale_factor, -end_y * scale_factor)
                    line.setPen(pen)
                self.view.setScene(self.scene)

    # Saves gcode file to your computer
    def saveGCode(self):
        if self.gcode_browser.toPlainText() == "":
            self.errorMessage("GCode empty")
        else:
            filename, _ = QFileDialog.getSaveFileName(self, filter="*.cnc")
            if filename:
                f = open(filename, 'w')
                toWrite = ""
                toWrite = toWrite.join(''.join(self.gcode_browser.toPlainText()) )
                f.write(toWrite)
                self.setWindowTitle(str(os.path.basename(filename)) + " - Notepad Alpha")
                f.close()
        
    def errorMessage(self, message):
        self.msg = QMessageBox()
        self.msg.setWindowTitle("Error")
        self.msg.setText(message)
        self.msg.exec_()
       
def main():
    app = QApplication(sys.argv)
    qdarktheme.setup_theme()
    window = DXFParserGUI()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0